<a href="https://colab.research.google.com/github/AlanItzep/DL/blob/master/save_and_restore_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q h5py pyyaml
!pip install -q tf_nightly

     |████████████████████████████████| 102.0MB 55.6MB/s 
     |████████████████████████████████| 501kB 39.9MB/s 
     |████████████████████████████████| 3.9MB 37.8MB/s 
     |████████████████████████████████| 61kB 20.6MB/s 


In [4]:
from __future__ import absolute_import,division,print_function,unicode_literals
import os

import tensorflow as tf
from tensorflow import keras
tf.__version__

'1.15.0-dev20190718'

In [0]:
(train_images, train_labels), (test_images,test_labels)=tf.keras.datasets.mnist.load_data()
train_labels=train_labels[:1000]
test_labels= test_labels[:1000]

train_images=train_images[:1000].reshape(-1,28*28)/255.0
test_images=test_images[:1000].reshape(-1,28*28)/255.0

In [20]:
def create_model():
  model=tf.keras.models.Sequential([
      keras.layers.Dense(512,activation=tf.keras.activations.relu, input_shape=(784,)),
      keras.layers.Dropout(0.2),
      keras.layers.Dense(10,activation=tf.keras.activations.softmax)
  ])
  
  model.compile(optimizer=tf.keras.optimizers.Adam(),
               loss=tf.keras.losses.sparse_categorical_crossentropy,
               metrics=['accuracy'])
  
  return model

model=create_model()
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [23]:
checkpoint_path="training_1/cp.ckpt"
checkpoint_dir=os.path.dirname(checkpoint_path)

cp_callback=tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                              save_weights_only=True,
                                              verbose=1)

model=create_model()
model.fit(train_images,train_labels,epochs=10,
         validation_data=(test_images,test_labels),
         callbacks=[cp_callback])

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 832/1000 [=======================>......] - ETA: 0s - loss: 1.3329 - acc: 0.5938
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 602us/sample - loss: 1.1975 - acc: 0.6420 - val_loss: 0.7687 - val_acc: 0.7580
Epoch 2/10
 832/1000 [=======================>......] - ETA: 0s - loss: 0.4451 - acc: 0.8786
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 290us/sample - loss: 0.4281 - acc: 0.8780 - val_loss: 0.5398 - val_acc: 0.8350
Epoch 3/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.2797 - acc: 0.9312
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 307us/sample - loss: 0.2882 - acc: 0.9280 - val_loss: 0.4611 - val_acc: 0.8500
Epoch 4/10
 864/1000 [========================>.....] - ETA: 0s - loss: 0.2415 - acc: 0.9375
Epoch 00004: saving model to training_1/cp.ckpt
1000/10

In [24]:
!ls {checkpoint_dir}

checkpoint		     cp.ckpt.data-00001-of-00002
cp.ckpt.data-00000-of-00002  cp.ckpt.index


In [25]:
model = create_model()

loss,acc=model.evaluate(test_images,test_labels)
print('Untrained model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 145us/sample - loss: 2.3259 - acc: 0.1230
Untrained model, accuracy: 12.30%


In [26]:
model.load_weights(checkpoint_path)
loss,acc=model.evaluate(test_images,test_labels)
print('Restored,accuracy: {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 75us/sample - loss: 0.4031 - acc: 0.8690
Restored,accuracy: 86.90%


In [27]:
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir=os.path.dirname(checkpoint_path)

cp_callback=tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path,verbose=1,save_weights_only=True,
    period=5
)

model=create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images,train_labels,
         epochs=50, callbacks=[cp_callback],
         validation_data=(test_images,test_labels),
         verbose=0)

W0718 15:57:16.210036 140682225497984 callbacks.py:861] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.



Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [30]:
! ls {checkpoint_dir}

checkpoint			  cp-0025.ckpt.data-00001-of-00002
cp-0000.ckpt.data-00000-of-00002  cp-0025.ckpt.index
cp-0000.ckpt.data-00001-of-00002  cp-0030.ckpt.data-00000-of-00002
cp-0000.ckpt.index		  cp-0030.ckpt.data-00001-of-00002
cp-0005.ckpt.data-00000-of-00002  cp-0030.ckpt.index
cp-0005.ckpt.data-00001-of-00002  cp-0035.ckpt.data-00000-of-00002
cp-0005.ckpt.index		  cp-0035.ckpt.data-00001-of-00002
cp-0010.ckpt.data-00000-of-00002  cp-0035.ckpt.index
cp-0010.ckpt.data-00001-of-00002  cp-0040.ckpt.data-00000-of-00002
cp-0010.ckpt.index		  cp-0040.ckpt.data-00001-of-00002
cp-0015.ckpt.data-00000-of-00002  cp-0040.ckpt.index
cp-0015.ckpt.data-00001-of-00002  cp-0045.ckpt.data-00000-of-00002
cp-0015.ckpt.index		  cp-0045.ckpt.data-00001-of-00002
cp-0020.ckpt.data-00000-of-00002  cp-0045.ckpt.index
cp-0020.ckpt.data-00001-of-00002  cp-0050.ckpt.data-00000-of-00002
cp-0020.ckpt.index		  cp-0050.ckpt.data-00001-of-00002
cp-0025.ckpt.data-00000-of-00002  cp-0050.ckpt.index


In [31]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [32]:
model = create_model()
model.load_weights(latest)
loss,acc=model.evaluate(test_images,test_labels)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

W0718 16:00:03.149975 140682225497984 util.py:144] Unresolved object in checkpoint: (root).optimizer.iter
W0718 16:00:03.151437 140682225497984 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_1
W0718 16:00:03.152461 140682225497984 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_2
W0718 16:00:03.153359 140682225497984 util.py:144] Unresolved object in checkpoint: (root).optimizer.decay
W0718 16:00:03.154330 140682225497984 util.py:144] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0718 16:00:03.155412 140682225497984 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-0.kernel
W0718 16:00:03.156703 140682225497984 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-0.bias
W0718 16:00:03.159595 140682225497984 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-1.kernel
W0

1000/1000 [==============================] - 0s 148us/sample - loss: 0.4846 - acc: 0.8730
Restored model, accuracy: 87.30%


In [33]:
model.save_weights('./checkpoints/my_checkpoint')

model=create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc=model.evaluate(test_images,test_labels)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 173us/sample - loss: 0.4846 - acc: 0.8730
Restored model, accuracy: 87.30%


In [35]:
model = create_model()
model.fit(train_images,train_labels,epochs=5)
model.save('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 268us/sample - loss: 1.2015 - acc: 0.6540
Epoch 2/5
1000/1000 [==============================] - 0s 161us/sample - loss: 0.4376 - acc: 0.8770
Epoch 3/5
1000/1000 [==============================] - 0s 150us/sample - loss: 0.2873 - acc: 0.9280
Epoch 4/5
1000/1000 [==============================] - 0s 153us/sample - loss: 0.2075 - acc: 0.9480
Epoch 5/5
1000/1000 [==============================] - 0s 165us/sample - loss: 0.1455 - acc: 0.9690


In [36]:
new_model=keras.models.load_model('my_model.h5')
new_model.summary()

W0718 16:07:55.871622 140682225497984 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0718 16:07:55.874563 140682225497984 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_18 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_37 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [39]:
loss, acc = new_model.evaluate(test_images,test_labels)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 89us/sample - loss: 0.4180 - acc: 0.8710
Restored model, accuracy: 87.10%


In [40]:
model= create_model()
model.fit(train_images,train_labels,epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 280us/sample - loss: 1.1834 - acc: 0.6590
Epoch 2/5
1000/1000 [==============================] - 0s 159us/sample - loss: 0.4324 - acc: 0.8750
Epoch 3/5
1000/1000 [==============================] - 0s 151us/sample - loss: 0.3104 - acc: 0.9150
Epoch 4/5
1000/1000 [==============================] - 0s 164us/sample - loss: 0.2212 - acc: 0.9460
Epoch 5/5
1000/1000 [==============================] - 0s 161us/sample - loss: 0.1611 - acc: 0.9690


In [41]:
import time
saved_model_path="./saved_models/"+str(int(time.time()))
tf.contrib.saved_model.save_keras_model(model,saved_model_path)

W0718 16:12:04.331765 140682225497984 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0718 16:12:04.334240 140682225497984 deprecation.py:323] From <ipython-input-41-ecd852c0175b>:3: export_saved_model (from tensorflow.python.keras.saving.saved_model_experimental) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.
W0718 16:12:05.350410 140682225497984 util.py:144] Unresolved object in checkpoint: (root).optimizer.iter
W0718 16:12:05.351909 140682225497984 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_1

In [42]:
!ls {saved_model_path}

assets	saved_model.pb	variables


In [43]:
new_model = tf.contrib.saved_model.load_keras_model(saved_model_path)
new_model.summary()

W0718 16:13:39.265690 140682225497984 deprecation.py:323] From <ipython-input-43-f5ba023fc0b4>:1: load_from_saved_model (from tensorflow.python.keras.saving.saved_model_experimental) is deprecated and will be removed in a future version.
Instructions for updating:
The experimental save and load functions have been  deprecated. Please switch to `tf.keras.models.load_model`.


Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_38 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_19 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_39 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [45]:
new_model.compile(optimizer=tf.keras.optimizers.Adam(),
                 loss=tf.keras.losses.sparse_categorical_crossentropy,
                 metrics=['accuracy'])
loss,acc=new_model.evaluate(test_images,test_labels)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 196us/sample - loss: 0.4153 - acc: 0.8710
Restored model, accuracy: 87.10%
